In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import sqlite3 as db

In [2]:
from pandasql import sqldf

In [3]:
# Points to remember
#The main function used in pandasql is sqldf. 
#sqldf accepts 2 parametrs

# a sql query string
# a set of session/environment variables (locals() or globals())


In [4]:
#define the global function, as managing local and global environment might be confusing at times
pysqldf = lambda q: sqldf(q, globals())

In [5]:
pysqldf

<function __main__.<lambda>>

In [6]:
#pandasql uses SQLite syntax. 

#Any pandas dataframes will be automatically detected by pandasql. 

#You can query them as you would any regular SQL table.

In [7]:
from pandasql import sqldf, load_meat, load_births

In [8]:
pysqldf = lambda q: sqldf(q, globals())

In [9]:
meat = load_meat()

In [10]:
meat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 827 entries, 0 to 826
Data columns (total 8 columns):
date               827 non-null datetime64[ns]
beef               827 non-null float64
veal               827 non-null float64
pork               827 non-null float64
lamb_and_mutton    827 non-null float64
broilers           635 non-null float64
other_chicken      143 non-null float64
turkey             635 non-null float64
dtypes: datetime64[ns](1), float64(7)
memory usage: 51.8 KB


In [11]:
meat.head()

,date,beef,veal,pork,lamb_and_mutton,broilers,other_chicken,turkey
0,1944-01-01,751.0,85.0,1280.0,89.0,NaN,NaN,NaN
1,1944-02-01,713.0,77.0,1169.0,72.0,NaN,NaN,NaN
2,1944-03-01,741.0,90.0,1128.0,75.0,NaN,NaN,NaN
3,1944-04-01,650.0,89.0,978.0,66.0,NaN,NaN,NaN
4,1944-05-01,681.0,106.0,1029.0,78.0,NaN,NaN,NaN


In [12]:
births = load_births()

In [13]:
births.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 408 entries, 0 to 407
Data columns (total 2 columns):
date      408 non-null datetime64[ns]
births    408 non-null int64
dtypes: datetime64[ns](1), int64(1)
memory usage: 6.5 KB


In [14]:
births.head()

,date,births
0,1975-01-01,265775
1,1975-02-01,241045
2,1975-03-01,268849
3,1975-04-01,247455
4,1975-05-01,254545


In [15]:
pysqldf("SELECT * FROM meat LIMIT 5;")

,date,beef,veal,pork,lamb_and_mutton,broilers,other_chicken,turkey
0,1944-01-01 00:00:00.000000,751.0,85.0,1280.0,89.0,None,None,None
1,1944-02-01 00:00:00.000000,713.0,77.0,1169.0,72.0,None,None,None
2,1944-03-01 00:00:00.000000,741.0,90.0,1128.0,75.0,None,None,None
3,1944-04-01 00:00:00.000000,650.0,89.0,978.0,66.0,None,None,None
4,1944-05-01 00:00:00.000000,681.0,106.0,1029.0,78.0,None,None,None


In [16]:
pysqldf("SELECT * FROM births LIMIT 5;")

,date,births
0,1975-01-01 00:00:00.000000,265775
1,1975-02-01 00:00:00.000000,241045
2,1975-03-01 00:00:00.000000,268849
3,1975-04-01 00:00:00.000000,247455
4,1975-05-01 00:00:00.000000,254545


In [17]:
#let's work on the joins and aggregations

In [18]:
# execution of simple group by query
q = """SELECT
            strftime('%Y', date) as year
           , SUM(beef) as beef_total
           FROM
              meat
           GROUP BY
              year
              LIMIT 5;"""

In [19]:
print(q)

SELECT
            strftime('%Y', date) as year
           , SUM(beef) as beef_total
           FROM
              meat
           GROUP BY
              year
              LIMIT 5;


In [20]:
pysqldf(q)

,year,beef_total
0,1944,8801.0
1,1945,9936.0
2,1946,9010.0
3,1947,10096.0
4,1948,8766.0


In [21]:
# you want to print all the results and then do selection of top 10 records
q = """SELECT
            strftime('%Y', date) as year
           , SUM(beef) as beef_total
           FROM
              meat
           GROUP BY
              year;"""

In [22]:
pysqldf(q).head(10)

,year,beef_total
0,1944,8801.0
1,1945,9936.0
2,1946,9010.0
3,1947,10096.0
4,1948,8766.0
5,1949,9142.0
6,1950,9248.0
7,1951,8549.0
8,1952,9337.0
9,1953,12055.0


In [23]:
# order by using random functions
q = """SELECT 
    *
    FROM
        meat
    ORDER BY RANDOM()
    LIMIT 10;"""


In [24]:
print(q)

SELECT 
    *
    FROM
        meat
    ORDER BY RANDOM()
    LIMIT 10;


In [25]:
print (pysqldf(q))

                         date    beef  veal    pork  lamb_and_mutton  \
0  2008-08-01 00:00:00.000000  2266.8  11.2  1803.9             13.3   
1  1969-04-01 00:00:00.000000  1667.0  52.0  1145.0             46.0   
2  1981-08-01 00:00:00.000000  1828.0  33.0  1157.0             25.0   
3  1982-07-01 00:00:00.000000  1802.0  34.0  1040.0             27.0   
4  1977-12-01 00:00:00.000000  2045.0  63.0  1108.0             25.0   
5  1980-06-01 00:00:00.000000  1725.0  30.0  1312.0             22.0   
6  1953-03-01 00:00:00.000000   931.0  94.0   809.0             63.0   
7  1966-11-01 00:00:00.000000  1650.0  72.0  1094.0             50.0   
8  1994-04-01 00:00:00.000000  1901.0  22.0  1432.0             27.0   
9  2000-02-01 00:00:00.000000  2175.0  18.0  1558.0             20.0   

   broilers  other_chicken  turkey  
0    3080.5           47.9   506.9  
1     533.5            NaN    21.0  
2    1003.8            NaN   261.7  
3    1029.5            NaN   228.3  
4     753.2           

In [26]:
#executing union all statements
q = """
        SELECT
            date
            , 'beef' AS meat_type
            , beef AS value
        FROM meat
        UNION ALL
        SELECT
            date
            , 'veal' AS meat_type
            , veal AS value
        FROM meat
        
        UNION ALL
        
        SELECT
            date
            , 'pork' AS meat_type
            , pork AS value
        FROM meat
        UNION ALL
        SELECT
            date
            , 'lamb_and_mutton' AS meat_type
            , lamb_and_mutton AS value
        FROM meat
        ORDER BY 1
    """

In [27]:
print(q)


        SELECT
            date
            , 'beef' AS meat_type
            , beef AS value
        FROM meat
        UNION ALL
        SELECT
            date
            , 'veal' AS meat_type
            , veal AS value
        FROM meat
        
        UNION ALL
        
        SELECT
            date
            , 'pork' AS meat_type
            , pork AS value
        FROM meat
        UNION ALL
        SELECT
            date
            , 'lamb_and_mutton' AS meat_type
            , lamb_and_mutton AS value
        FROM meat
        ORDER BY 1
    


In [28]:
print (pysqldf(q).head(10))

                         date        meat_type   value
0  1944-01-01 00:00:00.000000             beef   751.0
1  1944-01-01 00:00:00.000000             veal    85.0
2  1944-01-01 00:00:00.000000             pork  1280.0
3  1944-01-01 00:00:00.000000  lamb_and_mutton    89.0
4  1944-02-01 00:00:00.000000             beef   713.0
5  1944-02-01 00:00:00.000000             veal    77.0
6  1944-02-01 00:00:00.000000             pork  1169.0
7  1944-02-01 00:00:00.000000  lamb_and_mutton    72.0
8  1944-03-01 00:00:00.000000             beef   741.0
9  1944-03-01 00:00:00.000000             veal    90.0


In [29]:
q = """
    SELECT
        m.*
        , b.births
    FROM
        meat m
    INNER JOIN
        births b
            on m.date = b.date
    ORDER BY
        m.date;
"""

In [30]:
print(q)


    SELECT
        m.*
        , b.births
    FROM
        meat m
    INNER JOIN
        births b
            on m.date = b.date
    ORDER BY
        m.date;



In [31]:
print(pysqldf(q).head())

                         date    beef  veal    pork  lamb_and_mutton  \
0  1975-01-01 00:00:00.000000  2106.0  59.0  1114.0             36.0   
1  1975-02-01 00:00:00.000000  1845.0  50.0   954.0             31.0   
2  1975-03-01 00:00:00.000000  1891.0  57.0   976.0             35.0   
3  1975-04-01 00:00:00.000000  1895.0  60.0  1100.0             34.0   
4  1975-05-01 00:00:00.000000  1849.0  59.0   934.0             31.0   

   broilers  other_chicken  turkey  births  
0     646.2            NaN    64.9  265775  
1     570.2            NaN    47.1  241045  
2     616.6            NaN    54.4  268849  
3     688.3            NaN    68.7  247455  
4     690.1            NaN    81.9  254545  


In [32]:
#Perform inner join on two tables
q = """SELECT
        m.date, m.beef, b.births
     FROM
        meat m
     INNER JOIN
        births b
           ON m.date = b.date;"""

In [33]:
print(q)

SELECT
        m.date, m.beef, b.births
     FROM
        meat m
     INNER JOIN
        births b
           ON m.date = b.date;


In [34]:
#after join, print first 10 records
pysqldf(q).head(10)

,date,beef,births
0,1975-01-01 00:00:00.000000,2106.0,265775
1,1975-02-01 00:00:00.000000,1845.0,241045
2,1975-03-01 00:00:00.000000,1891.0,268849
3,1975-04-01 00:00:00.000000,1895.0,247455
4,1975-05-01 00:00:00.000000,1849.0,254545
5,1975-06-01 00:00:00.000000,1849.0,254096
6,1975-07-01 00:00:00.000000,1916.0,275163
7,1975-08-01 00:00:00.000000,1961.0,281300
8,1975-09-01 00:00:00.000000,2065.0,270738
9,1975-10-01 00:00:00.000000,2270.0,265494


In [35]:
# use queries within queries
q = """
    SELECT 
        m1.date
        , m1.beef 
    FROM 
        meat m1 
    WHERE m1.date IN 
        (SELECT 
            date 
        FROM meat 
        WHERE 
            beef >= broilers 
        ORDER BY date)
"""

In [36]:
print(q)


    SELECT 
        m1.date
        , m1.beef 
    FROM 
        meat m1 
    WHERE m1.date IN 
        (SELECT 
            date 
        FROM meat 
        WHERE 
            beef >= broilers 
        ORDER BY date)



In [37]:
pysqldf(q).head()

,date,beef
0,1960-01-01 00:00:00.000000,1196.0
1,1960-02-01 00:00:00.000000,1089.0
2,1960-03-01 00:00:00.000000,1201.0
3,1960-04-01 00:00:00.000000,1066.0
4,1960-05-01 00:00:00.000000,1202.0


In [38]:
from sklearn.datasets import load_iris

In [148]:
import re

In [149]:
iris = load_iris()

In [150]:
#IRIS is now in numpy array format
iris.data[0:5,]

array([[ 5.1,  3.5,  1.4,  0.2],
       [ 4.9,  3. ,  1.4,  0.2],
       [ 4.7,  3.2,  1.3,  0.2],
       [ 4.6,  3.1,  1.5,  0.2],
       [ 5. ,  3.6,  1.4,  0.2]])

In [151]:
# convert IRIS to a data frame object
iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)

In [152]:
iris_df.head(5)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [153]:
# join the target variable to the iris df
iris_df['species'] = pd.Categorical.from_codes(iris.target, iris.target_names)

In [154]:
iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [155]:
# replace the '(cm)' in the column header to NULL/blank to clean the feature names
iris_df.columns = [re.sub("[() ]", "", col) for col in iris_df.columns]


In [156]:
iris_df.head()

,sepallengthcm,sepalwidthcm,petallengthcm,petalwidthcm,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [157]:
# print first 10 records from the iris df table
print(sqldf("SELECT * FROM iris_df LIMIT 10;"))

   sepallengthcm  sepalwidthcm  petallengthcm  petalwidthcm species
0            5.1           3.5            1.4           0.2  setosa
1            4.9           3.0            1.4           0.2  setosa
2            4.7           3.2            1.3           0.2  setosa
3            4.6           3.1            1.5           0.2  setosa
4            5.0           3.6            1.4           0.2  setosa
5            5.4           3.9            1.7           0.4  setosa
6            4.6           3.4            1.4           0.3  setosa
7            5.0           3.4            1.5           0.2  setosa
8            4.4           2.9            1.4           0.2  setosa
9            4.9           3.1            1.5           0.1  setosa


In [158]:
# select sepal width column and species column and print first 10 records
print(sqldf("SELECT sepalwidthcm, species FROM iris_df LIMIT 10;"))

   sepalwidthcm species
0           3.5  setosa
1           3.0  setosa
2           3.2  setosa
3           3.1  setosa
4           3.6  setosa
5           3.9  setosa
6           3.4  setosa
7           3.4  setosa
8           2.9  setosa
9           3.1  setosa


In [159]:
# let's compute some statistical functions from the table
# compute the mean, minimum and maximum values of sepal width feature and group by the species category


In [160]:
# the query to perform the above task would look like as below
q = """
      select
        species
        , avg(sepalwidthcm)
        , min(sepalwidthcm)
        , max(sepalwidthcm)
      from
        iris_df
      group by
        species;
        
"""

In [161]:
print(q)


      select
        species
        , avg(sepalwidthcm)
        , min(sepalwidthcm)
        , max(sepalwidthcm)
      from
        iris_df
      group by
        species;
        



In [162]:
print(sqldf(q))

      species  avg(sepalwidthcm)  min(sepalwidthcm)  max(sepalwidthcm)
0      setosa              3.418                2.3                4.4
1  versicolor              2.770                2.0                3.4
2   virginica              2.974                2.2                3.8


In [163]:
#let's create a new table using inner join on the iris df by creating two dummy tables a and b and join by species column
q = """
    select
        a.*
    from
        iris_df a
    inner join
        iris_df b
            on a.species = b.species
    limit 10;
"""

In [164]:
print(q)


    select
        a.*
    from
        iris_df a
    inner join
        iris_df b
            on a.species = b.species
    limit 10;



In [165]:
print(pysqldf(q))

   sepallengthcm  sepalwidthcm  petallengthcm  petalwidthcm species
0            5.1           3.5            1.4           0.2  setosa
1            5.1           3.5            1.4           0.2  setosa
2            5.1           3.5            1.4           0.2  setosa
3            5.1           3.5            1.4           0.2  setosa
4            5.1           3.5            1.4           0.2  setosa
5            5.1           3.5            1.4           0.2  setosa
6            5.1           3.5            1.4           0.2  setosa
7            5.1           3.5            1.4           0.2  setosa
8            5.1           3.5            1.4           0.2  setosa
9            5.1           3.5            1.4           0.2  setosa


In [166]:
# use a filter criteria to select some records where the condition is met

In [167]:
q = """
    select
        *
    from
        iris_df
    where
        species = 'virginica'
        and sepallengthcm > 7.7;
"""

In [168]:
print(q)


    select
        *
    from
        iris_df
    where
        species = 'virginica'
        and sepallengthcm > 7.7;



In [169]:
print(pysqldf(q))

   sepallengthcm  sepalwidthcm  petallengthcm  petalwidthcm    species
0            7.9           3.8            6.4           2.0  virginica


In [170]:
# create one unique id column in iris df
iris_df['id'] = range(len(iris_df))

In [171]:
iris_df.head()

,sepallengthcm,sepalwidthcm,petallengthcm,petalwidthcm,species,id
0,5.1,3.5,1.4,0.2,setosa,0
1,4.9,3.0,1.4,0.2,setosa,1
2,4.7,3.2,1.3,0.2,setosa,2
3,4.6,3.1,1.5,0.2,setosa,3
4,5.0,3.6,1.4,0.2,setosa,4


In [172]:
# fetch the records where a logical condition is met
q = """
    select
        *
    from
        iris_df
    where
        id in (select id from iris_df where sepalwidthcm*sepallengthcm > 25);
"""

In [173]:
print(q)


    select
        *
    from
        iris_df
    where
        id in (select id from iris_df where sepalwidthcm*sepallengthcm > 25);



In [174]:
print(pysqldf(q))

   sepallengthcm  sepalwidthcm  petallengthcm  petalwidthcm    species   id
0            5.7           4.4            1.5           0.4     setosa   15
1            7.2           3.6            6.1           2.5  virginica  109
2            7.7           3.8            6.7           2.2  virginica  117
3            7.9           3.8            6.4           2.0  virginica  131


In [83]:
#sqldf() provides a convenient interface of running SQL statement on data frames. 
#Similarly, Python also offers multiple ways to interact between SQL and Pandas DataFrames by 
#leveraging the lightweight SQLite engine.

#import the library SQLite
import sqlite3


In [17]:
# generate some sample numbers
def fib(n):
    a, b = 0, 1

    for _ in range(n):
        yield a
        a, b = b, a + b


In [18]:
#establish a conenction
connection = sqlite3.connect(':memory:')
cursor = connection.cursor()


In [19]:
# check if the DB name exists
with connection:
    cursor.execute('''CREATE TABLE IF NOT EXISTS fib (
                            calculated_value INTEGER)''')
    cursor.executemany('INSERT INTO fib VALUES (?)',
                       [(str(x),) for x in fib(10)])


In [20]:
#run a sample query
cursor.execute('SELECT * FROM fib')
print(cursor.fetchall())

[(0,), (1,), (1,), (2,), (3,), (5,), (8,), (13,), (21,), (34,)]


In [14]:
connection.close()

In [23]:
conn = sqlite3.connect(':memory:')
cur = conn.cursor()
#design a table in the sqlite db
sql = """
    create table sensor (
        sid         integer primary key not null,
        name        text,
        notes       text
    );"""

_ = cur.execute(sql)

In [25]:
print(sql)


    create table sensor (
        sid         integer primary key not null,
        name        text,
        notes       text
    );


In [26]:
sql = "insert into sensor(sid, name, notes) values (%d, '%s', '%s');"

In [27]:
print(sql)

insert into sensor(sid, name, notes) values (%d, '%s', '%s');


In [28]:
for (uid, name, notes) in [(201, 'Alpha', 'Sensor for weight'), \
                           (202, 'Beta', 'Sensor for conductivity'),
                           (203, 'Gamma', 'Sensor for surface oxides'),
                           (204, 'Delta', 'Sensor for length'),
                           (205, 'Epsilon', 'Sensor for x-ray'),
                           (206, 'Zeta', 'Color checker 9000'),
                           (207, 'Eta', 'Ultra-violet detector'), ]:
    cur.execute(sql % (uid, name, notes))


In [29]:
print(sql)

insert into sensor(sid, name, notes) values (%d, '%s', '%s');


In [53]:
import numpy as np
from collections import defaultdict
import gc
import time
from pandas import DataFrame
from pandas.util.testing import rands
import random


In [54]:
N = 10000

In [56]:
indices = np.array([rands(10) for _ in range(N)], dtype='O')
indices2 = np.array([rands(10) for _ in range(N)], dtype='O')
key = np.tile(indices[:8000], 10)
key2 = np.tile(indices2[:8000], 10)

In [57]:
left = DataFrame({'key' : key, 'key2':key2,
                  'value' : np.random.randn(80000)})

In [59]:
left.head()

,key,key2,value
0,YBOnfoSeQl,S1yHvLhEvt,-0.024912
1,Iq2HVee4P1,bP5DAe3xA8,0.926543
2,Jawh8RlJ6I,a2W0lGTv6t,-0.616354
3,fMibW63YLw,qUK5LlIXom,-0.222069
4,cFZUwReCpb,YVtc87ixLx,-1.034871


In [60]:
right = DataFrame({'key': indices[2000:], 'key2':indices2[2000:],
                   'value2' : np.random.randn(8000)})


In [61]:
right.head()

,key,key2,value2
0,8GDq1LbK8Q,Riobsv5WIz,-0.983521
1,KRh4t1HBVk,4nP5qnpLdw,-0.710309
2,NB8bhTas7x,IRHnka1FVh,1.050322
3,xL2IYzW6bB,XMeyWpwrab,0.792583
4,xLD0Ihjru5,Q5OY980ovi,0.707779


In [62]:
right2 = right.append(right, ignore_index=True)
right = right2


In [63]:
right.head()

,key,key2,value2
0,8GDq1LbK8Q,Riobsv5WIz,-0.983521
1,KRh4t1HBVk,4nP5qnpLdw,-0.710309
2,NB8bhTas7x,IRHnka1FVh,1.050322
3,xL2IYzW6bB,XMeyWpwrab,0.792583
4,xLD0Ihjru5,Q5OY980ovi,0.707779


In [64]:
right2.head()

,key,key2,value2
0,8GDq1LbK8Q,Riobsv5WIz,-0.983521
1,KRh4t1HBVk,4nP5qnpLdw,-0.710309
2,NB8bhTas7x,IRHnka1FVh,1.050322
3,xL2IYzW6bB,XMeyWpwrab,0.792583
4,xLD0Ihjru5,Q5OY980ovi,0.707779


In [65]:
random.shuffle(key2)

In [66]:
indices2 = indices.copy()

In [67]:
random.shuffle(indices2)

In [68]:
# Prepare Database
import sqlite3
create_sql_indexes = False

In [69]:
conn = sqlite3.connect(':memory:')

In [70]:
conn.execute('create table left( key varchar(10), key2 varchar(10), value int);')

In [71]:
conn.execute('create table right( key varchar(10), key2 varchar(10), value2 int);')

In [72]:
conn.executemany('insert into left values (?, ?, ?)',
                 zip(key, key2, left['value']))

In [73]:
conn.executemany('insert into right values (?, ?, ?)',
                 zip(right['key'], right['key2'], right['value2']))


In [74]:
# Create Indices
if create_sql_indexes:
    conn.execute('create index left_ix on left(key, key2)')
    conn.execute('create index right_ix on right(key, key2)')


In [75]:
join_methods = ['inner', 'left outer', 'left'] # others not supported

In [76]:
join_methods

['inner', 'left outer', 'left']

In [77]:
sql_results = DataFrame(index=join_methods, columns=[False])

In [78]:
sql_results

,False
inner,NaN
left outer,NaN
left,NaN


In [79]:
niter = 5

In [81]:
for sort in [False]:
    for join_method in join_methods:
        sql = """select *
        from left
           %s join right
             on left.key=right.key
               and left.key2 = right.key2""" % join_method
        if sort:
            sql = '%s order by key, key2' % sql
        f = lambda: list(conn.execute(sql)) # list fetches results
        g = lambda: conn.execute(sql) # list fetches results
        gc.disable()
        start = time.time()
        for _ in range(niter):
            f()
        elapsed = (time.time() - start) / niter
        gc.enable()
        sql_results[sort][join_method] = elapsed
sql_results.columns = ['sqlite3'] # ['dont_sort', 'sort']
sql_results.index = ['inner', 'outer', 'left']

In [82]:
sql_results

,sqlite3
inner,0.165884
outer,0.409653
left,0.425291


In [84]:
#advantages of pysqldf over pandasql are two-fold. 

#1. pysqldf is 2 – 3 times faster than pandasql. 
#2. pysqldf supports new function definitions, which is not available in pandasql. 

#However, the generic python interface to an in-memory SQLite database can be more efficient and 
#flexible than both pysqldf and pandasql

In [109]:
import sqlalchemy

In [110]:
sqlalchemy.__version__

'1.1.13'

#The SQLAlchemy Object Relational Mapper presents a method of associating user-defined Python classes with database tables, and instances of those classes (objects) with rows in their corresponding tables. It includes a system that transparently synchronizes all changes in state between objects and their related rows, called a unit of work, as well as a system for expressing database queries in terms of the user defined classes and their defined relationships between each other.

In [111]:
from sqlalchemy import create_engine

In [112]:
engine = create_engine('sqlite:///:memory:', echo=True)

In [113]:
#The echo flag is a shortcut to setting up SQLAlchemy logging, which is accomplished via Python’s standard logging module. 

In [114]:
#The return value of create_engine() is an instance of Engine, and it represents the core interface to the database, 
#adapted through a dialect that handles the details of the database and DBAPI in use. In this case the SQLite dialect will 
#interpret instructions to the Python built-in sqlite3 module.



In [115]:
from sqlalchemy.ext.declarative import declarative_base

In [116]:
Base = declarative_base()

In [117]:
from sqlalchemy import Column, Integer, String

In [118]:
class User(Base):
...     __tablename__ = 'users'
...
...     id = Column(Integer, primary_key=True)
...     name = Column(String)
...     fullname = Column(String)
...     password = Column(String)
...
...     def __repr__(self):
...        return "<User(name='%s', fullname='%s', password='%s')>" % (
...                             self.name, self.fullname, self.password)

In [120]:
#With our User class constructed via the Declarative system, we have defined information about our table,
#known as table metadata. The object used by SQLAlchemy to represent this information for a specific table 
#is called the Table object, 
#and here Declarative has made one for us.

In [122]:
User.__table__ 

Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), Column('password', String(), table=<users>), schema=None)

#The MetaData is a registry which includes the ability to emit a limited set of schema generation commands to the database. As our SQLite database does not actually have a users table present, we can use MetaData to issue CREATE TABLE statements to the database for all tables that don’t yet exist. 

In [123]:
Base.metadata.create_all(engine)

2018-03-18 07:28:37,666 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1


INFO:sqlalchemy.engine.base.Engine:SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1


2018-03-18 07:28:37,768 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


2018-03-18 07:28:37,774 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1


...

INFO:sqlalchemy.engine.base.Engine:
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	password VARCHAR, 
	PRIMARY KEY (id)
)




2018-03-18 07:28:37,799 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


2018-03-18 07:28:37,803 INFO sqlalchemy.engine.base.Engine COMMIT


INFO:sqlalchemy.engine.base.Engine:COMMIT


In [124]:
ed_user = User(name='ed', fullname='Ed Jones', password='edspassword')

In [125]:
ed_user.name

'ed'

In [126]:
ed_user.password

'edspassword'

In [127]:
str(ed_user.id)

'None'

In [128]:
from sqlalchemy.orm import sessionmaker

In [129]:
Session = sessionmaker(bind=engine)

In [130]:
#In the case where your application does not yet have an Engine when you define your module-level objects

In [131]:
Session = sessionmaker()

In [132]:
#when you create your engine with create_engine(), connect it to the Session using configure()

In [133]:
Session.configure(bind=engine) 

In [134]:
session = Session()

In [135]:
#Adding and Updating Objects

In [136]:
ed_user = User(name='ed', fullname='Ed Jones', password='edspassword')

In [137]:
session.add(ed_user)

In [138]:
our_user = session.query(User).filter_by(name='ed').first() 

2018-03-18 07:32:42,998 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.base.Engine:BEGIN (implicit)


2018-03-18 07:32:43,005 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)


INFO:sqlalchemy.engine.base.Engine:INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)


2018-03-18 07:32:43,007 INFO sqlalchemy.engine.base.Engine ('ed', 'Ed Jones', 'edspassword')


INFO:sqlalchemy.engine.base.Engine:('ed', 'Ed Jones', 'edspassword')


2018-03-18 07:32:43,013 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?


INFO:sqlalchemy.engine.base.Engine:SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?


2018-03-18 07:32:43,015 INFO sqlalchemy.engine.base.Engine ('ed', 1, 0)


INFO:sqlalchemy.engine.base.Engine:('ed', 1, 0)


In [139]:
our_user

<User(name='ed', fullname='Ed Jones', password='edspassword')>

In [140]:
ed_user is our_user

True

In [141]:
#We can add more User objects at once using add_all():

In [142]:
session.add_all([
...     User(name='wendy', fullname='Wendy Williams', password='foobar'),
...     User(name='mary', fullname='Mary Contrary', password='xxg527'),
...     User(name='fred', fullname='Fred Flinstone', password='blah')])

In [143]:
#the password for Ed isn’t too secure, so lets change it:

In [144]:
ed_user.password = 'f8s7ccs'

In [145]:
session.dirty

IdentitySet([<User(name='ed', fullname='Ed Jones', password='f8s7ccs')>])

In [146]:
session.new

IdentitySet([<User(name='wendy', fullname='Wendy Williams', password='foobar')>, <User(name='mary', fullname='Mary Contrary', password='xxg527')>, <User(name='fred', fullname='Fred Flinstone', password='blah')>])

#We tell the Session that we’d like to issue all remaining changes to the database and commit the transaction, which has been in progress throughout. We do this via commit(). 

In [147]:
session.commit()

2018-03-18 07:35:15,263 INFO sqlalchemy.engine.base.Engine UPDATE users SET password=? WHERE users.id = ?


INFO:sqlalchemy.engine.base.Engine:UPDATE users SET password=? WHERE users.id = ?


2018-03-18 07:35:15,269 INFO sqlalchemy.engine.base.Engine ('f8s7ccs', 1)


INFO:sqlalchemy.engine.base.Engine:('f8s7ccs', 1)


2018-03-18 07:35:15,272 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)


...

INFO:sqlalchemy.engine.base.Engine:INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)


2018-03-18 07:35:15,287 INFO sqlalchemy.engine.base.Engine ('fred', 'Fred Flinstone', 'blah')


INFO:sqlalchemy.engine.base.Engine:('fred', 'Fred Flinstone', 'blah')


2018-03-18 07:35:15,290 INFO sqlalchemy.engine.base.Engine COMMIT


INFO:sqlalchemy.engine.base.Engine:COMMIT


In [148]:
#commit() flushes the remaining changes to the database, and commits the transaction. 

In [149]:
#Since the Session works within a transaction, we can roll back changes made too.

In [150]:
ed_user.name = 'Edwardo'

In [151]:
fake_user = User(name='fakeuser', fullname='Invalid', password='12345')

In [152]:
session.add(fake_user)

In [153]:
session.query(User).filter(User.name.in_(['Edwardo', 'fakeuser'])).all()

2018-03-18 07:36:35,665 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.base.Engine:BEGIN (implicit)


2018-03-18 07:36:35,670 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.id = ?


INFO:sqlalchemy.engine.base.Engine:SELECT users.id AS users_id, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.id = ?


2018-03-18 07:36:35,673 INFO sqlalchemy.engine.base.Engine (1,)


...

2018-03-18 07:36:35,690 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name IN (?, ?)


INFO:sqlalchemy.engine.base.Engine:SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name IN (?, ?)


2018-03-18 07:36:35,693 INFO sqlalchemy.engine.base.Engine ('Edwardo', 'fakeuser')


INFO:sqlalchemy.engine.base.Engine:('Edwardo', 'fakeuser')


[<User(name='Edwardo', fullname='Ed Jones', password='f8s7ccs')>,
 <User(name='fakeuser', fullname='Invalid', password='12345')>]

In [154]:
session.rollback()

2018-03-18 07:36:49,342 INFO sqlalchemy.engine.base.Engine ROLLBACK


INFO:sqlalchemy.engine.base.Engine:ROLLBACK


In [155]:
ed_user.name

2018-03-18 07:36:56,242 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.base.Engine:BEGIN (implicit)


2018-03-18 07:36:56,247 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.id = ?


INFO:sqlalchemy.engine.base.Engine:SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.id = ?


2018-03-18 07:36:56,250 INFO sqlalchemy.engine.base.Engine (1,)


INFO:sqlalchemy.engine.base.Engine:(1,)


'ed'

In [156]:
fake_user in session

False

In [157]:
session.query(User).filter(User.name.in_(['ed', 'fakeuser'])).all()

2018-03-18 07:37:20,117 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name IN (?, ?)


INFO:sqlalchemy.engine.base.Engine:SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name IN (?, ?)


2018-03-18 07:37:20,121 INFO sqlalchemy.engine.base.Engine ('ed', 'fakeuser')


INFO:sqlalchemy.engine.base.Engine:('ed', 'fakeuser')


[<User(name='ed', fullname='Ed Jones', password='f8s7ccs')>]

#A Query object is created using the query() method on Session. This function takes a variable number of arguments, which can be any combination of classes and class-instrumented descriptors.

In [158]:
for instance in session.query(User).order_by(User.id):
...     print(instance.name, instance.fullname)

2018-03-18 07:37:41,250 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users ORDER BY users.id


INFO:sqlalchemy.engine.base.Engine:SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users ORDER BY users.id


2018-03-18 07:37:41,255 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


ed Ed Jones
wendy Wendy Williams
mary Mary Contrary
fred Fred Flinstone


In [159]:
for name, fullname in session.query(User.name, User.fullname):
...     print(name, fullname)

2018-03-18 07:38:26,315 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name, users.fullname AS users_fullname 
FROM users


INFO:sqlalchemy.engine.base.Engine:SELECT users.name AS users_name, users.fullname AS users_fullname 
FROM users


2018-03-18 07:38:26,319 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


ed Ed Jones
wendy Wendy Williams
mary Mary Contrary
fred Fred Flinstone


In [160]:
for row in session.query(User, User.name).all():
...    print(row.User, row.name)

2018-03-18 07:38:46,126 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users


INFO:sqlalchemy.engine.base.Engine:SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users


2018-03-18 07:38:46,130 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


<User(name='ed', fullname='Ed Jones', password='f8s7ccs')> ed
<User(name='wendy', fullname='Wendy Williams', password='foobar')> wendy
<User(name='mary', fullname='Mary Contrary', password='xxg527')> mary
<User(name='fred', fullname='Fred Flinstone', password='blah')> fred


In [161]:
for row in session.query(User.name.label('name_label')).all():
...    print(row.name_label)

2018-03-18 07:38:55,685 INFO sqlalchemy.engine.base.Engine SELECT users.name AS name_label 
FROM users


INFO:sqlalchemy.engine.base.Engine:SELECT users.name AS name_label 
FROM users


2018-03-18 07:38:55,688 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


ed
wendy
mary
fred


In [163]:
#The name given to a full entity such as User, assuming that multiple entities are present in the call to query(), 
#can be controlled using aliased() :


In [164]:
from sqlalchemy.orm import aliased

In [165]:
user_alias = aliased(User, name='user_alias')

In [166]:
for row in session.query(user_alias, user_alias.name).all():
...    print(row.user_alias)

2018-03-18 07:39:48,124 INFO sqlalchemy.engine.base.Engine SELECT user_alias.id AS user_alias_id, user_alias.name AS user_alias_name, user_alias.fullname AS user_alias_fullname, user_alias.password AS user_alias_password 
FROM users AS user_alias


INFO:sqlalchemy.engine.base.Engine:SELECT user_alias.id AS user_alias_id, user_alias.name AS user_alias_name, user_alias.fullname AS user_alias_fullname, user_alias.password AS user_alias_password 
FROM users AS user_alias


2018-03-18 07:39:48,128 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


<User(name='ed', fullname='Ed Jones', password='f8s7ccs')>
<User(name='wendy', fullname='Wendy Williams', password='foobar')>
<User(name='mary', fullname='Mary Contrary', password='xxg527')>
<User(name='fred', fullname='Fred Flinstone', password='blah')>


In [167]:
#Basic operations with Query include issuing LIMIT and OFFSET, most conveniently using Python array slices and 
#typically in conjunction with ORDER BY:


In [168]:
for u in session.query(User).order_by(User.id)[1:3]:
...    print(u)

2018-03-18 07:40:26,264 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users ORDER BY users.id
 LIMIT ? OFFSET ?


INFO:sqlalchemy.engine.base.Engine:SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users ORDER BY users.id
 LIMIT ? OFFSET ?


2018-03-18 07:40:26,268 INFO sqlalchemy.engine.base.Engine (2, 1)


INFO:sqlalchemy.engine.base.Engine:(2, 1)


<User(name='wendy', fullname='Wendy Williams', password='foobar')>
<User(name='mary', fullname='Mary Contrary', password='xxg527')>


In [169]:
for name, in session.query(User.name).\
...             filter_by(fullname='Ed Jones'):
...    print(name)

2018-03-18 07:40:40,993 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name 
FROM users 
WHERE users.fullname = ?


INFO:sqlalchemy.engine.base.Engine:SELECT users.name AS users_name 
FROM users 
WHERE users.fullname = ?


2018-03-18 07:40:40,998 INFO sqlalchemy.engine.base.Engine ('Ed Jones',)


INFO:sqlalchemy.engine.base.Engine:('Ed Jones',)


ed


In [170]:
for name, in session.query(User.name).\
...             filter(User.fullname=='Ed Jones'):
...    print(name)

2018-03-18 07:40:51,774 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name 
FROM users 
WHERE users.fullname = ?


INFO:sqlalchemy.engine.base.Engine:SELECT users.name AS users_name 
FROM users 
WHERE users.fullname = ?


2018-03-18 07:40:51,779 INFO sqlalchemy.engine.base.Engine ('Ed Jones',)


INFO:sqlalchemy.engine.base.Engine:('Ed Jones',)


ed


In [171]:
for user in session.query(User).\
...          filter(User.name=='ed').\
...          filter(User.fullname=='Ed Jones'):
...    print(user)

2018-03-18 07:41:12,290 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ? AND users.fullname = ?


INFO:sqlalchemy.engine.base.Engine:SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ? AND users.fullname = ?


2018-03-18 07:41:12,295 INFO sqlalchemy.engine.base.Engine ('ed', 'Ed Jones')


INFO:sqlalchemy.engine.base.Engine:('ed', 'Ed Jones')


<User(name='ed', fullname='Ed Jones', password='f8s7ccs')>


In [172]:
#Here’s a rundown of some of the most common operators used in filter():


In [175]:
query = session.query(User).filter(User.name.like('%ed')).order_by(User.id)

In [176]:
query.all() #all() returns a list:

2018-03-18 07:42:48,471 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name LIKE ? ORDER BY users.id


INFO:sqlalchemy.engine.base.Engine:SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name LIKE ? ORDER BY users.id


2018-03-18 07:42:48,475 INFO sqlalchemy.engine.base.Engine ('%ed',)


INFO:sqlalchemy.engine.base.Engine:('%ed',)


[<User(name='ed', fullname='Ed Jones', password='f8s7ccs')>,
 <User(name='fred', fullname='Fred Flinstone', password='blah')>]

In [177]:
query.first()

2018-03-18 07:43:15,732 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name LIKE ? ORDER BY users.id
 LIMIT ? OFFSET ?


INFO:sqlalchemy.engine.base.Engine:SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name LIKE ? ORDER BY users.id
 LIMIT ? OFFSET ?


2018-03-18 07:43:15,736 INFO sqlalchemy.engine.base.Engine ('%ed', 1, 0)


INFO:sqlalchemy.engine.base.Engine:('%ed', 1, 0)


<User(name='ed', fullname='Ed Jones', password='f8s7ccs')>

In [179]:
from sqlalchemy import text

In [180]:
#Literal strings can be used flexibly with Query, by specifying their use with the text() construct

In [181]:
for user in session.query(User).\
...             filter(text("id<224")).\
...             order_by(text("id")).all():
...     print(user.name)

2018-03-18 07:44:24,145 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE id<224 ORDER BY id


INFO:sqlalchemy.engine.base.Engine:SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE id<224 ORDER BY id


2018-03-18 07:44:24,149 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


ed
wendy
mary
fred


In [182]:
session.query(User).filter(text("id<:value and name=:name")).\
...     params(value=224, name='fred').order_by(User.id).one()

2018-03-18 07:44:41,027 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE id<? and name=? ORDER BY users.id


INFO:sqlalchemy.engine.base.Engine:SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE id<? and name=? ORDER BY users.id


2018-03-18 07:44:41,034 INFO sqlalchemy.engine.base.Engine (224, 'fred')


INFO:sqlalchemy.engine.base.Engine:(224, 'fred')


<User(name='fred', fullname='Fred Flinstone', password='blah')>

In [184]:
#To use an entirely string-based statement, a text() construct representing a complete 
#statement can be passed to from_statement(). 

In [183]:
session.query(User).from_statement(
...                     text("SELECT * FROM users where name=:name")).\
...                     params(name='ed').all()

2018-03-18 07:44:58,385 INFO sqlalchemy.engine.base.Engine SELECT * FROM users where name=?


INFO:sqlalchemy.engine.base.Engine:SELECT * FROM users where name=?


2018-03-18 07:44:58,389 INFO sqlalchemy.engine.base.Engine ('ed',)


INFO:sqlalchemy.engine.base.Engine:('ed',)


[<User(name='ed', fullname='Ed Jones', password='f8s7ccs')>]

In [185]:
stmt = text("SELECT name, id, fullname, password "
...             "FROM users where name=:name")

In [186]:
stmt = stmt.columns(User.name, User.id, User.fullname, User.password)

In [187]:
session.query(User).from_statement(stmt).params(name='ed').all()

2018-03-18 07:46:00,195 INFO sqlalchemy.engine.base.Engine SELECT name, id, fullname, password FROM users where name=?


INFO:sqlalchemy.engine.base.Engine:SELECT name, id, fullname, password FROM users where name=?


2018-03-18 07:46:00,200 INFO sqlalchemy.engine.base.Engine ('ed',)


INFO:sqlalchemy.engine.base.Engine:('ed',)


[<User(name='ed', fullname='Ed Jones', password='f8s7ccs')>]

In [188]:
#The count() method is used to determine how many rows the SQL statement would return. 

In [189]:
session.query(User).filter(User.name.like('%ed')).count()

2018-03-18 07:46:33,417 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name LIKE ?) AS anon_1


INFO:sqlalchemy.engine.base.Engine:SELECT count(*) AS count_1 
FROM (SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name LIKE ?) AS anon_1


2018-03-18 07:46:33,422 INFO sqlalchemy.engine.base.Engine ('%ed',)


INFO:sqlalchemy.engine.base.Engine:('%ed',)


2

In [190]:
from sqlalchemy import func

In [191]:
session.query(func.count(User.name), User.name).group_by(User.name).all()

2018-03-18 07:46:50,858 INFO sqlalchemy.engine.base.Engine SELECT count(users.name) AS count_1, users.name AS users_name 
FROM users GROUP BY users.name


INFO:sqlalchemy.engine.base.Engine:SELECT count(users.name) AS count_1, users.name AS users_name 
FROM users GROUP BY users.name


2018-03-18 07:46:50,862 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


[(1, 'ed'), (1, 'fred'), (1, 'mary'), (1, 'wendy')]

In [193]:
#To achieve our simple SELECT count(*) FROM table, we can apply it as:

In [194]:
session.query(func.count('*')).select_from(User).scalar()

2018-03-18 07:47:20,611 INFO sqlalchemy.engine.base.Engine SELECT count(?) AS count_1 
FROM users


INFO:sqlalchemy.engine.base.Engine:SELECT count(?) AS count_1 
FROM users


2018-03-18 07:47:20,616 INFO sqlalchemy.engine.base.Engine ('*',)


INFO:sqlalchemy.engine.base.Engine:('*',)


4

In [195]:
session.query(func.count(User.id)).scalar()

2018-03-18 07:47:31,757 INFO sqlalchemy.engine.base.Engine SELECT count(users.id) AS count_1 
FROM users


INFO:sqlalchemy.engine.base.Engine:SELECT count(users.id) AS count_1 
FROM users


2018-03-18 07:47:31,761 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


4